# Current solution uses multiprocessing in Python to create the matrix

Here is the list of things to benchmark:

- shapely vs wkt 
- matrix containing qgisGeometry ==> update vlayer using enumerate rather than dict 


# Wkt vs Shapely point 
To measure : 
. size in memory of matrix
. Time to build matrix 
. Time for ONF to create the new geometry


### Wkt vs Shapely point

Does the wkt transformation takes longer than the if checks to see if Point is empty in ONF ? 


### Matrix with  QgsGeometry

This is somewhat cumbersome to implement : requires passing the Pyqgis path to the worker nodes to create the QgsGeometries



# ONF Shapely Point and QgsPointXY

```python
from shapely.geometry import Point
import time

Timer = 0
n = 20000
item = Point(0,0)
for i in range(n) :        
    now = time.time()
    x,y = item.x, item.y
    geom = QgsGeometry.fromPointXY(QgsPointXY(x,y))
    Timer += time.time() - now 
    
print(f"time over {n} : {Timer} s, average {Timer/n}")
```


Using WKT code :

```python
from shapely.geometry import Point
import time

Timer = 0
n = 20000
item = Point(0,0).wkt
for i in range(n) :        
    now = time.time()
    geom = QgsGeometry().fromWkt(item)
    Timer += time.time() - now 
    
print(f"time over {n} : {Timer} s, average {Timer/n}")

```

Results 

QgsPointXY:
exec(Path('/tmp/tmp5m0u71g5').read_text())
time over 20000 : 0.6315727233886719 s, average 3.1578636169433594e-05

Wkt: 

exec(Path('/tmp/tmp300ld8yj').read_text())
time over 20000 : 0.06986188888549805 s, average 3.4930944442749022e-06


Advantage for WKT 

# Matrix Shapely point vs WKT


TODO

# Removing datetime object update in ONF


Using these parameters :

. 0.2 % of boats
. tdelta size : 60 frames
. 150 frames animation



Before with single datetime attribute, requiring the update of the datetime at each frame of the temporal controller

tt.get_average_fps()
60.72075001983026

After adding a start-end datetime attribute field removing the attribute update at each ONF call

tt.get_average_fps()
71.28473220962775


TLDR :

-A noticeable improvement in ONF time 

# ONF FPS for N objects

Limit on ONF FPS optimization, how much time would it take to update geometries for x objects with x = [100, 1000, 5000, 10000, 15000, 20000]


### Method






In [ ]:
from datetime import datetime
import time

canvas = iface.mapCanvas()
temporalController = canvas.temporalController()

def create_vlayer(name, num_fields, frames):
    vlayer = QgsVectorLayer("Point", name, "memory")
    pr = vlayer.dataProvider()
    pr.addAttributes([QgsField("start_time", QVariant.DateTime), QgsField("end_time", QVariant.DateTime)])
    vlayer.updateFields()
    tp = vlayer.temporalProperties()
    tp.setIsActive(True)
    tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeStartAndEndFromFields)
    tp.setStartField("start_time")
    tp.setEndField("end_time")

    features_list =[]
    start_datetime_obj = QDateTime(datetime(2023,6,1,0,0,0))
    end_datetime_obj = QDateTime(datetime(2023,6,1,23,0,0))
    for i in range(num_fields):
        feat = QgsFeature(vlayer.fields())
        feat.setAttributes([start_datetime_obj, end_datetime_obj])
        x,y = (1,1)
        geom = QgsGeometry.fromPointXY(QgsPointXY(x,y))
        feat.setGeometry(geom)
        features_list.append(feat)

    QgsProject.instance().addMapLayer(vlayer)
    
    
    vlayer.startEditing()
    vlayer.addFeatures(features_list)
    vlayer.commitChanges()
    iface.vectorLayerTools().stopEditing(vlayer)

    current_time_stamp_column = ["POINT (1 1)"]*num_fields

    fps_record = []
    for i in range(frames):
        start_time = time.time()    

        new_geometries = {}  # Dictionary {feature_id: QgsGeometry}
        for i in range(num_fields): #TODO : compare vs Nditer
            new_geometries[i] = QgsGeometry.fromWkt(f"POINT ({i} {i})")


        vlayer.startEditing()
        # self.qviz.vlayer.dataProvider().changeAttributeValues(attribute_changes) # Updating attribute values for all features
        vlayer.dataProvider().changeGeometryValues(new_geometries) # Updating geometries for all features
        vlayer.commitChanges()
        iface.vectorLayerTools().stopEditing(vlayer)


        end_time = time.time()
        fps_record.append(1/(end_time-start_time))

    average_fps = sum(fps_record)/len(fps_record)
    
    return average_fps

x = [100, 1000, 5000, 10000, 15000, 20000, 25000, 30000, 350000]
frames = 100
for i, x in enumerate(x):
    print(f"Average fps for {x} features over {frames} frames : ")
    print(create_vlayer(f"points_{i}", x, frames))
    print("")

In [7]:
import numpy as np

array = np.arange(1, 5000)
array 

array([   1,    2,    3, ..., 4997, 4998, 4999])

In [11]:
%%timeit
a = [{i : j} for i,j  in enumerate(array)]

1.57 ms ± 93.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit
b = {}
for i, j in enumerate(array):
    b[i] = j

893 µs ± 8.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
